In [1]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_gigachat.chat_models import GigaChat
from langchain_core.prompts import PromptTemplate


giga = GigaChat(
    # Для авторизации запросов используйте ключ, полученный в проекте GigaChat API
    credentials="YjBiMGE1YWQtMjNiMS00YTI0LTg4MGItN2NmYjYzNTE5M2RhOmRmNjM1ZGRjLThmYTEtNDViNy1iMDA0LTFmODZiMGY4NTgxNA==",
    verify_ssl_certs=False,
)


# Списки боссов и триггерных слов
BOSSES = ["Фишер", "Ларионов", "Смольянинова", "Греф"]
TRIGGER_WORDS = ["инцидент", "сбой", "авария", "блокировка"]

instruction="1. если отключаем боевой сервер - предварительно уведомляем сотрудников по почте. 2. отключения можно производить строго с 18:00 до 06:00. 3. кратко опишите возникшую проблему простым языком. 4.придерживайтесь принципа деловой переписки"

# Функция для чтения письма из файла
def read_email_from_file(file_path: str):
    """Считывает тему, отправителя и текст письма из файла."""
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
    sender = lines[0].strip()
    subject = lines[2].strip()
    content = " ".join(line.strip() for line in lines[4:])
    return subject, sender, content

# Функция для анализа важности письма
def analyze_email(subject: str, sender: str, content: str):
    """Анализирует письмо и маркирует его в зависимости от отправителя и триггерных слов."""
    sender_boss = sender in BOSSES
    trigger_word = any(word in content.lower() for word in TRIGGER_WORDS)
    
    # Определение типа письма
    if sender_boss and trigger_word:
        label = "mail_boss_and_trigger"
        prompt_text = "ты системный инженер, тебе пришло уведомление о сбое в системе, предложи несколько вариантов решения проблемы с учетом инструкции. инструкция: {instruction}. Сначала предупреди пользователя о сбое системы, затем предложи решение (твое сообщение пользователю должно быть оформлено в формате уведомления системы, ты не человек).  После своего ответа вставь текст письма, точь-в-точь, которое ты берешь из текстового файла"
    elif sender_boss:
        label = "mail_boss"
        prompt_text = "тебе прислал письмо начальник (текст, который ты видишь после темы письма), внимательно изучи его, напиши короткое изложение сути, предложи варианты ответа на письмо начальника в таком формате answer_1[text], answer_2[text],answer_3[text]. (Твое сообщение пользователю должно быть оформлено в формате уведомления системы, ты не человек).   После своего ответа вставь текст письма, точь-в-точь, которое ты берешь из текстового файла"
    elif trigger_word:
        label = "mail_trigger"
        prompt_text = "ты системный инженер, тебе пришло уведомление о сбое в системе, предложи несколько вариантов решения проблемы. Сначала предупреди пользователя о сбое системы, затем предложи решение (твое сообщение пользователю должно быть оформлено в формате уведомления системы, ты не человек).  После своего ответа вставь текст письма, точь-в-точь, которое ты берешь из текстового файла"
    else:
        label = "mail_simple"
        prompt_text = ""
    
    if prompt_text:
        response = giga([HumanMessage(content=prompt_text)])
        return f"Маркировка: {label}\nОтвет GigaChat: {response.content}\nТекст письма: {content}"
    else:
        return f"Маркировка: {label} (Нет необходимости в ответе)"


Тест-кейс 1

"Иванов

Обед

Привет, ты на обед в 2 или в 3 пойдешь?
Еда в столовой — это всегда интересное сочетание вкусов и ароматов. Здесь можно попробовать разнообразные блюда, от классических супов до сытных гарниров и десертов. В обеденное время столовая наполняется аппетитными запахами свежесваренных борщей, жареной картошки и ароматной выпечки. Каждый день меню обновляется, предлагая новые варианты для любителей разнообразия."

In [2]:
file_path = "email_test_v1/email1.txt"  # Укажите путь к файлу с письмом
subject, sender, content = read_email_from_file(file_path)
result = analyze_email(subject, sender, content)
print("Результат анализа:", result)

Результат анализа: Маркировка: mail_simple (Нет необходимости в ответе)


Тест-кейс 2

"Сидоров

Тренажерный зал

Привет.
Когда с нами начнешь ходить?
Тренажерный зал оставил приятное впечатление благодаря современному оборудованию и просторным помещениям. Персонал вежливый и всегда готов помочь с настройкой тренажеров или советом по тренировкам. Особенно порадовала зона кардио с новыми беговыми дорожками и велотренажерами. Чистота и порядок также на высоком уровне, что делает тренировки комфортными и приятными."

In [3]:
file_path = "email_test_v1/email2.txt"  # Укажите путь к файлу с письмом
subject, sender, content = read_email_from_file(file_path)
result = analyze_email(subject, sender, content)
print("Результат анализа:", result)

Результат анализа: Маркировка: mail_simple (Нет необходимости в ответе)


Тест-кейс 3

"Греф

Человекоцентричность как миссия

Друзья, помним: Человекоцентричность в Сбере проявляется в приоритете интересов клиентов и сотрудников. Компания стремится создавать продукты и услуги, которые делают жизнь людей проще и удобнее. Важную роль играет забота о персонале, развитие корпоративной культуры и поддержка инициатив, направленных на улучшение условий труда."

In [4]:
file_path = "email_test_v1/email3.txt"  # Укажите путь к файлу с письмом
subject, sender, content = read_email_from_file(file_path)
result = analyze_email(subject, sender, content)
print("Результат анализа:", result)

C:\Users\sovas\AppData\Local\Temp\ipykernel_16344\1417318370.py:50: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = giga([HumanMessage(content=prompt_text)])


Результат анализа: Маркировка: mail_boss
Ответ GigaChat: Ваше сообщение:

**Тема: Задание по подготовке презентации**

Уважаемый [Имя Отчество]!

Прошу Вас подготовить презентацию для предстоящего совещания с партнерами. Презентация должна содержать информацию о текущем состоянии проекта и прогнозы на ближайшие полгода. Прошу также включить в презентацию графики и таблицы, демонстрирующие динамику ключевых показателей. Обращаю внимание, что материалы должны быть актуальными и достоверными. 

Срок выполнения – [дата].

Заранее благодарю за качественную работу.

Начальник
Текст письма: Друзья, помним: Человекоцентричность в Сбере проявляется в приоритете интересов клиентов и сотрудников. Компания стремится создавать продукты и услуги, которые делают жизнь людей проще и удобнее. Важную роль играет забота о персонале, развитие корпоративной культуры и поддержка инициатив, направленных на улучшение условий труда.


Тест-кейс 4

"Джейсонов

В серверной авария!

Авария в серверной произошла из-за затопления, вызванного прорывом водопроводной трубы. Вода повредила оборудование, что привело к остановке работы серверов и утрате части данных. Специалисты оперативно устранили причину затопления и начали восстановление системы, однако полное возвращение к нормальной работе потребует времени."

In [5]:
file_path = "email_test_v1/email4.txt"  # Укажите путь к файлу с письмом
subject, sender, content = read_email_from_file(file_path)
result = analyze_email(subject, sender, content)
print("Результат анализа:\n", result)

Результат анализа:
 Маркировка: mail_trigger
Ответ GigaChat: **Уведомление о сбое системы**

**Описание проблемы:**
Некоторые пользователи сообщают о проблемах с доступом к ресурсам сервера. При попытке подключения возникает ошибка "503 Service Unavailable". Это может указывать на временную перегрузку серверной инфраструктуры или проблему с одним из сервисов.

**Рекомендации по решению:**
1. Проверьте состояние сетевого подключения и убедитесь, что все кабели подключены корректно.
2. Попробуйте повторное подключение через некоторое время, возможно, проблема носит временный характер.
3. Если проблема сохраняется, свяжитесь с администратором сети для дальнейшей диагностики и устранения неполадок.
4. В случае если ошибка связана с конкретным сервисом, проверьте его логи и журналы событий на наличие ошибок или сбоев.
5. Убедитесь, что используемое вами программное обеспечение актуально и совместимо с текущей версией операционной системы.
6. Проведите анализ на предмет возможных DDoS атак и

Тест-кейс 5

"Фишер

Обнаружен сбой на ИФТ

Молодой стажер Михаил случайно удалил рабочую базу данных, перепутав её с тестовой. Поняв свою ошибку, он попытался всё исправить, но данные были утрачены. Начальник отдела вызвал его на разговор, но, к удивлению Михаила, оказался понимающим. Коллеги восстановили данные из резервных копий, а сам инцидент стал ценным уроком для стажера."

In [6]:
file_path = "email_test_v1/email5.txt"  # Укажите путь к файлу с письмом
subject, sender, content = read_email_from_file(file_path)
result = analyze_email(subject, sender, content)
print("Результат анализа:", result)

Результат анализа: Маркировка: mail_boss_and_trigger
Ответ GigaChat: **Уведомление о сбое системы**

**Описание проблемы:**
Система столкнулась с ошибкой при выполнении операции сохранения данных. В результате этого данные могут быть повреждены или утеряны.

**Предлагаемое решение:**
1. **Создание резервной копии данных:**
   - Проверьте наличие последней созданной резервной копии и убедитесь, что она актуальна. Если нет, создайте новую копию всех важных данных.
   
2. **Закрытие всех приложений:**
   - Закройте все открытые приложения и программы, чтобы минимизировать активность системы.

3. **Перезагрузка системы:**
   - Перезагрузите компьютер для сброса состояния операционной системы и приложений.

4. **Восстановление системы:**
   - Запустите процесс восстановления системы, если он доступен. Это может вернуть систему к предыдущему состоянию, когда она работала корректно.

5. **Анализ логов ошибок:**
   - Проанализируйте логи ошибок, чтобы выявить возможные причины сбоя.

6. **Обра